# Predicción de Diabetes
## Hackathon HackUPM 2025 

**Pipeline completo:**
1. Cargar train.json + test.json
2. Extraer edad, género, features clínicos (con regex robustas y negaciones)
3. Generar embeddings con BioClinicalBERT (768-dim)
4. Agrupar por paciente (media de features + embeddings)
5. EDA completo sin errores
6. Modelado con RandomForest
7. Predicciones finales en test
8. Guardado en múltiples formatos

**Autores:**
**Fecha:** 03-11-2025


In [2]:
# INSTALACIONES (ejecutar si es primera vez, descomentar)
!pip install -q transformers torch pandas numpy tqdm scikit-learn
!pip install --upgrade git+https://github.com/huggingface/transformers.git
!pip install word2number

from word2number import w2n
import json
import pandas as pd
import numpy as np
import torch
import re
import warnings
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

warnings.filterwarnings('ignore')

print("Librerías importadas")
print(f"CUDA disponible: {torch.cuda.is_available()}") # CUDA es una plataforma de computación de Nvidia que ejecuta opers en las GPUs.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # prepara tu código para trabajar en la GPU si está disponible, o en la CPU si no lo está.
print(f"Dispositivo: {device}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.3 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 26.8 MB/s eta 0:00:00
ERROR: pip's dependency 

## 1️⃣ Cargar Datos (train.json + test.json)

In [3]:
print("\nLeyendo train.json...")
with open("/kaggle/input/data1222/train.json", "r") as f:
    train_data = json.load(f)

print("Leyendo test.json...")
with open("/kaggle/input/data1222/test.json", "r") as f:
    test_data = json.load(f)

# Crear DataFrames iniciales
df_train_raw = pd.DataFrame(train_data)
df_test_raw = pd.DataFrame(test_data)

print(f"\n Train: {len(df_train_raw)} registros, {df_train_raw.shape[1]} columnas")
print(f" Test: {len(df_test_raw)} registros, {df_test_raw.shape[1]} columnas")

print(f"\n Columnas : {df_train_raw.columns.tolist()}")

print(f"\n Distribución diabetes en TRAIN:")
print(df_train_raw["has_diabetes"].value_counts())
print(f"Proporción positivos: {df_train_raw['has_diabetes'].mean()}")


Leyendo train.json...
Leyendo test.json...

 Train: 3000 registros, 3 columnas
 Test: 300 registros, 2 columnas

 Columnas : ['patient_id', 'has_diabetes', 'medical_note']

 Distribución diabetes en TRAIN:
has_diabetes
0    2100
1     900
Name: count, dtype: int64
Proporción positivos: 0.3


## 2️⃣ Extraccion de features clinicos

In [4]:
outliers = {
    "age": [],
    "glucose": [],
    "hba1c": [],
    "bmi": []
}

def safe_float(x):
    """Convierte a float de forma segura."""
    try:
        return float(x)
    except:
        return np.nan

def extract_age(text):
    """Extrae edad con rango válido 0-120."""
    if not isinstance(text, str):
        return np.nan
    t = text.lower()
    # Patrón 1: "X-year-old" o "age X"
    m = re.search(r'(\d{1,3})\s*(?:year)?-?\s*(?:year-old|yr|years?\s*old)', t)
    age = None
    if not m:
        m = re.search(r'(?:age|aged)\s*(?:is)?\s*(\d{1,3})\b', t)
        if not m:
            m = re.search(r"\b([a-zA-Z]+(?:[-\s][a-zA-Z]+)*)-year-old\b", t)
            if m:
                age = w2n.word_to_num(m.group(1))
    if not age:
        age = int(m.group(1)) if m else np.nan
    if not m:
        outliers["age"].append(t)
    return age if (not np.isnan(age) and 0 <= age <= 120) else np.nan

def extract_gender(text):
    """Extrae género (male/female/unknown)."""
    if not isinstance(text, str):
        return "unknown"
    t = text.lower()
    male_count = len(re.findall(r'\b(?:male|man|he|his|him|boy)\b', t))
    female_count = len(re.findall(r'\b(?:female|woman|she|her|girl)\b', t))

    if male_count > 0 and female_count == 0:
        return "male"
    elif female_count > 0 and male_count == 0:
        return "female"
    elif male_count == female_count == 0:
        return "unknown"
    else:
        return "male" if male_count >= female_count else "female"

def extract_bmi(text):
    """Extrae BMI con rango válido 8-80."""
    if not isinstance(text, str):
        return np.nan
    t = text.lower()
    m = re.search(r'\b(?:bmi|imc)\b[^0-9]{0,30}(\d{1,3}(?:\.\d+)?)', t)
    if not m:
        m = re.search(r'\b(?:bmi|imc)\b.{0,26}range.{0,10}(\d{1,3}(?:[.,]\d+)?)', t)
    v = safe_float(m.group(1)) if m else np.nan
    if not m:
        outliers["bmi"].append(t)
    return v if (not np.isnan(v) and 0 <= v <= 80) else np.nan

def extract_hba1c(text):
    """Extrae HbA1c con rango válido 3-20."""
    if not isinstance(text, str):
        return np.nan
    t = text.lower()
    # Ventana local: hasta 20 chars después de "hba1c"
    m = re.search(r'(?:hba1c|a1c)[^0-9]{0,20}(\d{1,2}(?:\.\d+)?)\s*%?', t)
    if not m:
        pattern = re.compile(
            r'\b(?:hba1c(?: level)?s?\b.{0,10}(?:is|are|was|were|within|of)?\s*(very\s+high|high|elevated|normal|within normal limits|low)|(very\s+high|high|elevated|normal|within normal limits|low)\s+(?:levels\s+of\s+)?hba1c)\b',
            re.IGNORECASE
        )
        m = pattern.search(t)
        if m:
            mapping = {
                "normal": 5.5,
                "elevated": 6.3,
                "high": 6.5,
                "very high": 8
                }
            v = mapping[m.group(1) or m.group(2)] if m else np.nan
            return v if (not np.isnan(v) and 0 <= v <= 20) else np.nan
    if not m:
        outliers["hba1c"].append(t)
    
    v = safe_float(m.group(1)) if m else np.nan
    return v if (not np.isnan(v) and 0 <= v <= 20) else np.nan

def extract_glucose(text):
    """Extrae glucosa (aleatoria o postprandial) con rango válido 40-600."""
    if not isinstance(text, str):
        return np.nan
    t = text.lower()
    # Preferir "glucose" + 2-3 dígitos
    m = re.search(r'\bglucose\b[^0-9]{0,20}(\d{2,3})(?:\s*mg/dl)?', t)
    v = safe_float(m.group(1)) if m else np.nan
    if not m:
        pattern = re.compile(
            r'\b(?:'
            r'(?:\w+\s+)?glucose(?: (?:level|levels|measurement|measurements|reading|readings))?\b.{0,10}(?:is|are|was|were|within|of)?\s*(?P<val>very\s+high|high|elevated|normal|within normal limits|low|decreased|reduced|abnormal)'
            r'|'
            r'(?P<val2>very\s+high|high|elevated|normal|within normal limits|low|decreased|reduced|abnormal)\s+(?:\w+\s+)?glucose(?: (?:level|levels|measurement|measurements|reading|readings))?'
            r')\b',
            re.IGNORECASE
        )
        m = pattern.search(t)
        if m:
            mapping = {
                "low": 70,
                "normal": 140,
                "within normal limits": 140,
                "elevated": 165,
                "high": 200,
                "abnormal": 200,
                "very high": 250,
                }
            try:
                v = mapping[m.group(1) or m.group(2)] if m else np.nan
            except:
                print(t)
                exit(0)
            return v if (not np.isnan(v)) else np.nan
    if not m:
        outliers["glucose"].append(t)
    return v if (not np.isnan(v)) else np.nan

def extract_flags(text):
    """Extrae hipertensión, cardiopatía, fumación (respeta negaciones)."""
    if not isinstance(text, str):
        return 0, 0, "unknown"

    t = text.lower()
    NEG_PAT = r'(?:no\s+|without\s+|denies\s+|negative\s+for\s+|no\s+history\s+of\s+)'

    # Hipertensión: negación > pos
    hyp_neg = bool(re.search(NEG_PAT + r'(?:hypertension|high\s+blood\s+pressure)', t))
    hyp_pos = bool(re.search(r'\bhypertension\b|\bhigh\s+blood\s+pressure\b', t)) and not hyp_neg
    has_hypertension = 1 if hyp_pos else 0

    # Cardiopatía: negación > pos
    hd_neg = bool(re.search(NEG_PAT + r'(?:heart\s+disease|cardiovascular)', t))
    hd_pos = bool(re.search(r'\bheart\s+disease\b|\bcardiovascular', t)) and not hd_neg
    has_heart_disease = 1 if hd_pos else 0

    # Fumación
    if re.search(r'\bnon-smoker\b|\bnever\s+smoked\b', t):
        smoking = "never"
    elif re.search(r'\b(?:past|former)\s+(?:smoker|smoking)\b', t):
        smoking = "past"
    elif re.search(r'\bcurrent\s+smoker\b|\bis\s+a\s+smoker\b|\bsmoker\b', t):
        smoking = "current"
    else:
        smoking = "unknown"

    return has_hypertension, has_heart_disease, smoking

# TEST: verificar extracción en muestra
import numpy as np

for i in range(min(3, len(df_train_raw))):
    note = df_train_raw["medical_note"].iloc[i]
    age = extract_age(note)
    gender = extract_gender(note)
    bmi = extract_bmi(note)
    hba1c = extract_hba1c(note)
    glucose = extract_glucose(note)
    hyp, hd, smoking = extract_flags(note)

    age_s    = "NaN" if (age is None or (isinstance(age, float) and np.isnan(age))) else f"{int(age)}"
    bmi_s    = "NaN" if (bmi is None or np.isnan(bmi)) else f"{bmi:.1f}"
    hba1c_s  = "NaN" if (hba1c is None or np.isnan(hba1c)) else f"{hba1c:.1f}"
    glucose_s= "NaN" if (glucose is None or np.isnan(glucose)) else f"{glucose:.0f}"

    print(
        f"\n  [{i}] age={age_s}, gender={gender}, bmi={bmi_s}, "
        f"hba1c={hba1c_s}, glucose={glucose_s}, hyp={hyp}, hd={hd}, smoking={smoking}"
    )


print("\n✅ Extracción verificada")


  [0] age=16, gender=female, bmi=21.5, hba1c=6.2, glucose=140, hyp=0, hd=1, smoking=never

  [1] age=15, gender=female, bmi=33.6, hba1c=5.5, glucose=158, hyp=1, hd=1, smoking=unknown

  [2] age=54, gender=male, bmi=21.5, hba1c=5.5, glucose=145, hyp=0, hd=1, smoking=current

✅ Extracción verificada


In [5]:
# Aplicar a TRAIN
print("\n Train: extrayendo features...")
df_train_raw["age"] = df_train_raw["medical_note"].apply(extract_age)
df_train_raw["gender"] = df_train_raw["medical_note"].apply(extract_gender)
df_train_raw["bmi"] = df_train_raw["medical_note"].apply(extract_bmi)
df_train_raw["hba1c"] = df_train_raw["medical_note"].apply(extract_hba1c)
df_train_raw["glucose"] = df_train_raw["medical_note"].apply(extract_glucose)

tmp_train = df_train_raw["medical_note"].apply(extract_flags)
df_train_raw["has_hypertension"] = [t[0] for t in tmp_train]
df_train_raw["has_heart_disease"] = [t[1] for t in tmp_train]
df_train_raw["smoking_status"] = [t[2] for t in tmp_train]

# Aplicar a TEST
print(" Test: extrayendo features...")
df_test_raw["age"] = df_test_raw["medical_note"].apply(extract_age)
df_test_raw["gender"] = df_test_raw["medical_note"].apply(extract_gender)
df_test_raw["bmi"] = df_test_raw["medical_note"].apply(extract_bmi)
df_test_raw["hba1c"] = df_test_raw["medical_note"].apply(extract_hba1c)
df_test_raw["glucose"] = df_test_raw["medical_note"].apply(extract_glucose)

tmp_test = df_test_raw["medical_note"].apply(extract_flags)
df_test_raw["has_hypertension"] = [t[0] for t in tmp_test]
df_test_raw["has_heart_disease"] = [t[1] for t in tmp_test]
df_test_raw["smoking_status"] = [t[2] for t in tmp_test]

print("\n Features extraídos correctamente")
print(f"\n MUESTRA TRAIN (primeras 3 filas):")
cols = ['patient_id', 'age', 'gender', 'bmi', 'hba1c', 'glucose', 'smoking_status', 'has_diabetes']
print(df_train_raw[cols].head(3))


 Train: extrayendo features...
 Test: extrayendo features...

 Features extraídos correctamente

 MUESTRA TRAIN (primeras 3 filas):
   patient_id   age  gender    bmi  hba1c  glucose smoking_status  \
0       82555  16.0  female  21.49    6.2    140.0          never   
1       92299  15.0  female  33.62    5.5    158.0        unknown   
2       18725  54.0    male  21.46    5.5    145.0        current   

   has_diabetes  
0             0  
1             0  
2             0  


## 3️⃣ BioClinicalBERT: Generar Embeddings (768-dim)

In [6]:
def load_bioclinicalbert():
    """Carga Bio_ClinicalBERT desde HuggingFace."""
    model_name = "emilyalsentzer/Bio_ClinicalBERT" # coge el modelo de HuggingFace
    print(f" Cargando {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name) # convierte el texto en tokens
    model = AutoModel.from_pretrained(model_name) # Genera los embeddings desde la red a los token de la secuencia
    model.eval() # evalua que se ha pasado de texto a embeddings
    model.to(device) # mueve el modelo a GPU (kaggle)
    return tokenizer, model

def mean_pool(last_hidden_state, attention_mask):
    """Mean pooling con mask."""
    mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float() # mascara que ve cuales partes de los valores del vector influyeno o no 
    sum_hidden = (last_hidden_state * mask).sum(dim=1) # quita los insignificantes y sumas todos los buenos
    sum_mask = torch.clamp(mask.sum(dim=1), min=1e-9) # clump evita que se divida en la siguiente linea entre 0 
    return sum_hidden / sum_mask # calcula la media de todos los tokens entre los de la mascara

def embed_text(text, tokenizer, model, max_length=512):
    """Genera 1 embedding de 768 dims para un texto si esta vacia o no es texto."""
    if not isinstance(text, str) or len(text.strip()) == 0: 
        return np.zeros(768, dtype=np.float32) 

    tokens = tokenizer(text,padding=True,truncation=True,max_length=max_length,return_tensors="pt").to(device) # crea los tokens desde la frase

    with torch.no_grad(): # sin gradientes para no entrenar modelo
        output = model(**tokens) # crea los embeddings
        pooled = mean_pool(output.last_hidden_state, tokens["attention_mask"]) # llama a la funcion de la mascara para ponderar correctamente

    return pooled.cpu().numpy()[0].astype(np.float32)

# Cargar modelo (una sola vez)
tokenizer, model = load_bioclinicalbert()

# Generar embeddings TRAIN
print("\n Generando embeddings TRAIN...")
train_embeddings = []
for note in tqdm(df_train_raw["medical_note"].tolist(), desc="Train embeddings", total=len(df_train_raw)):
    emb = embed_text(note, tokenizer, model)
    train_embeddings.append(emb)

df_train_raw["embedding"] = train_embeddings

# Generar embeddings TEST
print("\n Generando embeddings TEST...")
test_embeddings = []
for note in tqdm(df_test_raw["medical_note"].tolist(), desc="Test embeddings", total=len(df_test_raw)):
    emb = embed_text(note, tokenizer, model)
    test_embeddings.append(emb)

df_test_raw["embedding"] = test_embeddings


 Cargando emilyalsentzer/Bio_ClinicalBERT...


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]


 Generando embeddings TRAIN...



Train embeddings: 100%|██████████| 3000/3000 [00:34<00:00, 87.86it/s]



 Generando embeddings TEST...


Test embeddings: 100%|██████████| 300/300 [00:03<00:00, 93.91it/s]


## 4️⃣ Agrupar por Paciente + Expandir Embeddings

In [7]:
def most_common(series):
    """Retorna valor más frecuente o 'unknown' del embedding."""
    s = series.dropna()
    return s.mode().iat[0] if not s.mode().empty else "unknown"

def emb_mean(series):
    """Promedia embeddings."""
    stacked = np.vstack(series.values)
    return stacked.mean(axis=0)


# Agregación TRAIN
print("\n Agrupando TRAIN por patient_id...")
agg_dict = {
    "medical_note": "count",
    "has_diabetes": "first",
    "age": "mean",
    "gender": most_common,
    "bmi": "mean",
    "hba1c": "mean",
    "glucose": "mean",
    "has_hypertension": "max",
    "has_heart_disease": "max",
    "smoking_status": most_common,
    "embedding": emb_mean
}

df_train_agg = df_train_raw.groupby("patient_id").agg(agg_dict).reset_index() # por si se repiten pacientes, agruparlos
df_train_agg.rename(columns={"medical_note": "note_count"}, inplace=True)

print(f" Train agrupado: {df_train_agg.shape[0]} pacientes únicos")

# Agregación TEST (sin has_diabetes)
print("\n Agrupando TEST por patient_id...")
agg_dict_test = {
    "medical_note": "count",
    "age": "mean",
    "gender": most_common,
    "bmi": "mean",
    "hba1c": "mean",
    "glucose": "mean",
    "has_hypertension": "max",
    "has_heart_disease": "max",
    "smoking_status": most_common,
    "embedding": emb_mean
}

df_test_agg = df_test_raw.groupby("patient_id").agg(agg_dict_test).reset_index()
df_test_agg.rename(columns={"medical_note": "note_count"}, inplace=True)

print(f" Test agrupado: {df_test_agg.shape[0]} pacientes únicos")

# Expandir embeddings en columnas
print("\n Expandiendo embeddings (768 columnas)...")

emb_train = np.vstack(df_train_agg["embedding"].values)
emb_test = np.vstack(df_test_agg["embedding"].values)

# Crea los embeddings (columnas) en el Dataframe
emb_cols = [f"emb_{i}" for i in range(emb_train.shape[1])]
emb_df_train = pd.DataFrame(emb_train, columns=emb_cols)
emb_df_test = pd.DataFrame(emb_test, columns=emb_cols)

df_train_final = pd.concat([df_train_agg.drop(columns=["embedding"]).reset_index(drop=True), emb_df_train], axis=1)
df_test_final = pd.concat([df_test_agg.drop(columns=["embedding"]).reset_index(drop=True), emb_df_test], axis=1)

print(f"\n Train final: {df_train_final.shape}")
print(f" Test final: {df_test_final.shape}")

print(f"\n PRIMERAS FILAS TRAIN (con age y gender):")
print(df_train_final[['patient_id', 'age', 'gender', 'bmi', 'hba1c', 'glucose', 'smoking_status', 'has_diabetes', 'note_count']].head(5))



 Agrupando TRAIN por patient_id...
 Train agrupado: 3000 pacientes únicos

 Agrupando TEST por patient_id...
 Test agrupado: 300 pacientes únicos

 Expandiendo embeddings (768 columnas)...

 Train final: (3000, 779)
 Test final: (300, 778)

 PRIMERAS FILAS TRAIN (con age y gender):
   patient_id   age  gender    bmi  hba1c  glucose smoking_status  \
0           5  23.0    male  21.05    6.5    200.0        current   
1          14  70.0  female  32.63    5.5    165.0        unknown   
2          36  42.0  female  31.50    5.8    200.0          never   
3          67  71.0    male  39.03    6.3      NaN          never   
4         127  66.0  female  23.58    5.8    145.0          never   

   has_diabetes  note_count  
0             0           1  
1             0           1  
2             0           1  
3             1           1  
4             1           1  


In [8]:
df_train_final

,patient_id,note_count,has_diabetes,age,gender,bmi,hba1c,glucose,has_hypertension,has_heart_disease,...,emb_758,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767
0,5,1,0,23.0,male,21.05,6.5,200.0,0,1,...,0.168776,-0.137289,-0.300718,0.055569,-0.012067,-0.191791,0.101528,0.040025,-0.036748,-0.125205
1,14,1,0,70.0,female,32.63,5.5,165.0,0,1,...,0.140446,-0.112927,-0.244349,-0.033866,0.026291,-0.076418,0.078653,0.214932,-0.077384,0.030703
2,36,1,0,42.0,female,31.50,5.8,200.0,0,1,...,0.144146,-0.082072,-0.269926,-0.027545,0.041318,-0.097104,0.003274,0.072266,-0.098089,-0.058813
3,67,1,1,71.0,male,39.03,6.3,NaN,0,1,...,0.158193,-0.121447,-0.211743,-0.014441,-0.079715,-0.143533,0.071336,0.013479,-0.085260,0.011425
4,127,1,1,66.0,female,23.58,5.8,145.0,0,1,...,0.054226,-0.093798,-0.332355,-0.064596,0.043048,-0.200588,0.136516,0.041142,-0.021674,-0.084981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,95954,1,0,61.0,male,24.52,6.6,165.0,0,1,...,0.172905,-0.028232,-0.289735,0.031709,-0.036546,-0.199922,0.011492,0.098776,-0.031317,-0.044507
2996,96030,1,0,73.0,female,25.07,6.3,158.0,0,1,...,0.050674,-0.060000,-0.283743,-0.085756,-0.056065,-0.177056,0.061681,0.067599,-0.042998,-0.052660
2997,96035,1,0,18.0,male,37.65,6.5,90.0,0,1,...,0.141482,-0.078641,-0.259597,0.134808,0.033681,-0.146596,0.002997,0.081718,0.014971,-0.069236
2998,96093,1,1,74.0,female,6.00,6.8,165.0,1,0,...,0.124343,0.006571,-0.255402,-0.014538,-0.036401,-0.155102,0.129003,0.092416,-0.071230,-0.012100


## 5️⃣ Análisis Exploratorio (EDA Completo)

In [9]:


print(f"\n EDAD (Age):")
print(f"   Media: {df_train_final['age'].mean():.1f} años")
print(f"   Mediana: {df_train_final['age'].median():.1f} años")
print(f"   Rango: {df_train_final['age'].min():.0f} - {df_train_final['age'].max():.0f} años")
print(f"   Faltantes: {df_train_final['age'].isna().sum()}")

print(f"\n GÉNERO (Gender):")
gen_dist = df_train_final['gender'].value_counts()
for g, c in gen_dist.items():
    print(f"   {g}: {c} ({c/len(df_train_final)*100:.1f}%)")

print(f"\n BMI:")
print(f"   Media: {df_train_final['bmi'].mean():.2f} ± {df_train_final['bmi'].std():.2f}")
print(f"   Rango: {df_train_final['bmi'].min():.2f} - {df_train_final['bmi'].max():.2f}")
print(f"   Faltantes: {df_train_final['bmi'].isna().sum()}")

print(f"\n HbA1c:")
print(f"   Media: {df_train_final['hba1c'].mean():.2f} ± {df_train_final['hba1c'].std():.2f}")
print(f"   Rango: {df_train_final['hba1c'].min():.2f} - {df_train_final['hba1c'].max():.2f}")
print(f"   Faltantes: {df_train_final['hba1c'].isna().sum()}")

print(f"\n GLUCOSA (Glucose):")
print(f"   Media: {df_train_final['glucose'].mean():.2f} ± {df_train_final['glucose'].std():.2f}")
print(f"   Rango: {df_train_final['glucose'].min():.2f} - {df_train_final['glucose'].max():.2f}")
print(f"   Faltantes: {df_train_final['glucose'].isna().sum()}")

print(f"\n HIPERTENSIÓN:")
hyp_count = df_train_final['has_hypertension'].sum()
print(f"   Con hipertensión: {hyp_count} ({hyp_count/len(df_train_final)*100:.1f}%)")

print(f"\n ENFERMEDAD CARDÍACA:")
hd_count = df_train_final['has_heart_disease'].sum()
print(f"   Con cardiopatía: {hd_count} ({hd_count/len(df_train_final)*100:.1f}%)")

print(f"\n FUMACIÓN:")
smoke_dist = df_train_final['smoking_status'].value_counts()
for s, c in smoke_dist.items():
    print(f"   {s}: {c} ({c/len(df_train_final)*100:.1f}%)")

print(f"\n DIABETES (TARGET):")
diab_dist = df_train_final['has_diabetes'].value_counts()
print(f"   Negativo (0): {diab_dist[0]} ({diab_dist[0]/len(df_train_final)*100:.1f}%)")
print(f"   Positivo (1): {diab_dist[1]} ({diab_dist[1]/len(df_train_final)*100:.1f}%)")

print(f"\n CORRELACIÓN CON DIABETES:")
numeric_cols = ['age', 'bmi', 'hba1c', 'glucose', 'has_hypertension', 'has_heart_disease', 'note_count']
corr = df_train_final[numeric_cols + ['has_diabetes']].corr()['has_diabetes'].sort_values(ascending=False)
print(corr.head(8))



 EDAD (Age):
   Media: 46.3 años
   Mediana: 49.0 años
   Rango: 1 - 80 años
   Faltantes: 9

 GÉNERO (Gender):
   female: 1663 (55.4%)
   male: 1337 (44.6%)

 BMI:
   Media: 28.05 ± 7.77
   Rango: 0.00 - 72.21
   Faltantes: 5

 HbA1c:
   Media: 6.25 ± 1.07
   Rango: 3.50 - 9.00
   Faltantes: 115

 GLUCOSA (Glucose):
   Media: 161.97 ± 43.13
   Rango: 15.00 - 300.00
   Faltantes: 211

 HIPERTENSIÓN:
   Con hipertensión: 1017 (33.9%)

 ENFERMEDAD CARDÍACA:
   Con cardiopatía: 2874 (95.8%)

 FUMACIÓN:
   never: 1566 (52.2%)
   unknown: 768 (25.6%)
   past: 552 (18.4%)
   current: 114 (3.8%)

 DIABETES (TARGET):
   Negativo (0): 2100 (70.0%)
   Positivo (1): 900 (30.0%)

 CORRELACIÓN CON DIABETES:
has_diabetes         1.000000
hba1c                0.535270
glucose              0.446588
age                  0.429397
bmi                  0.315766
has_hypertension     0.199608
has_heart_disease   -0.062372
note_count                NaN
Name: has_diabetes, dtype: float64


In [10]:
scaler_bmi = StandardScaler()
scaler_hba1c = StandardScaler()
scaler_glucose = StandardScaler()
scaler_age = StandardScaler()

df_train_final["bmi"] = scaler_bmi.fit_transform(df_train_final[["bmi"]])
df_train_final["hba1c"] = scaler_hba1c.fit_transform(df_train_final[["hba1c"]])
df_train_final["glucose"] = scaler_glucose.fit_transform(df_train_final[["glucose"]])
df_train_final["age"] = scaler_age.fit_transform(df_train_final[["age"]])

df_test_final["bmi"] = scaler_bmi.transform(df_test_final[["bmi"]])
df_test_final["hba1c"] = scaler_hba1c.transform(df_test_final[["hba1c"]])
df_test_final["glucose"] = scaler_glucose.transform(df_test_final[["glucose"]])
df_test_final["age"] = scaler_age.transform(df_test_final[["age"]])

## 6️⃣ Preparación para Modelado

In [11]:
# Preparar X_train e y_train
X_train = df_train_final.drop(columns=["patient_id", "has_diabetes"])
y_train = df_train_final["has_diabetes"]

# Preparar X_test
X_test = df_test_final.drop(columns=["patient_id"])

# Rellenar NaNs
print("\n Rellenando valores faltantes...")

for col in X_train.columns:
    if X_train[col].dtype == 'object':
        # Categóricos: usar moda
        mode_val = X_train[col].mode()[0] if not X_train[col].mode().empty else "unknown"
        X_train[col] = X_train[col].fillna(mode_val)
        X_test[col] = X_test[col].fillna(mode_val)
    else:
        # Numéricos: usar media
        mean_val = X_train[col].mean()
        X_train[col] = X_train[col].fillna(mean_val)
        X_test[col] = X_test[col].fillna(mean_val)

# One-hot encoding para categorías
print("\n One-hot encoding para gender y smoking_status...")
X_train = pd.get_dummies(X_train, columns=['gender', 'smoking_status'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['gender', 'smoking_status'], drop_first=True)

# Alinear columnas
for col in set(X_train.columns) - set(X_test.columns):
    X_test[col] = 0
for col in set(X_test.columns) - set(X_train.columns):
    X_train[col] = 0

X_train = X_train[sorted(X_train.columns)]
X_test = X_test[sorted(X_train.columns)]

print(f"\n X_train final shape: {X_train.shape}")
print(f" X_test final shape: {X_test.shape}")
print(f"\n Columnas features: {X_train.columns.tolist()[:15]}... (+{len(X_train.columns)-15})")



 Rellenando valores faltantes...

 One-hot encoding para gender y smoking_status...

 X_train final shape: (3000, 779)
 X_test final shape: (300, 779)

 Columnas features: ['age', 'bmi', 'emb_0', 'emb_1', 'emb_10', 'emb_100', 'emb_101', 'emb_102', 'emb_103', 'emb_104', 'emb_105', 'emb_106', 'emb_107', 'emb_108', 'emb_109']... (+764)


## 7️⃣ Ensemble con Weighted Soft Voting




In [24]:
# ===== LIBRERÍAS =====
try:
    from xgboost import XGBClassifier
    from catboost import CatBoostClassifier
except Exception:
    from xgboost import XGBClassifier
    from catboost import CatBoostClassifier

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import numpy as np

# ===== UTILIDADES =====
def auc_weights(models, X_tr, y_tr, X_val, y_val):
    aucs, names, fitted = [], [], []
    for name, model in models:
        model.fit(X_tr, y_tr)
        proba = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, proba)
        aucs.append(auc); names.append(name); fitted.append(model)
        print(f"{name}: AUC={auc:.4f}")
    auc_clipped = np.clip(aucs, 0.5, 1.0)
    raw = (np.array(auc_clipped) - 0.5) + 1e-6
    w = (raw / raw.sum()).tolist()
    print("Pesos:", {n: round(wi,3) for n, wi in zip(names, w)})
    return list(zip(names, fitted)), w

def report(name, model, X_val, y_val):
    y_hat = model.predict(X_val)
    y_pb  = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pb)
    f1  = f1_score(y_val, y_hat, zero_division=0)
    acc = accuracy_score(y_val, y_hat)
    print(f"[{name}] AUC={auc:.4f}  F1={f1:.4f}  Acc={acc:.4f}")

# ===== PREPARACIÓN DE LOS DATOS =====
# Aquí debes asegurarte de que X_train y y_train ya están preparados previamente en tu código

# Divide el conjunto de datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Ratio de desbalance
pos = max(1, int(y_train.sum()))
neg = int(len(y_train) - pos)
spw = neg / pos
print(f"scale_pos_weight={spw:.2f}")

# ===== META-MODELO PARA STACKING =====
meta_lr = LogisticRegression(max_iter=2000, solver='lbfgs', class_weight='balanced', C=0.5, random_state=42)

# ===== CATB + XGB + SVM =====
catb_opt = CatBoostClassifier(
    iterations=200, depth=6, learning_rate=0.05, l2_leaf_reg=3.0,
    loss_function='Logloss', eval_metric='Logloss', class_weights=[1.0, spw],
    random_state=42, verbose=0  # Menor número de iteraciones
)

xgb_opt = XGBClassifier(
    n_estimators=200, max_depth=5, learning_rate=0.03,
    subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
    eval_metric='logloss', tree_method='hist', n_jobs=-1, random_state=42,
    scale_pos_weight=spw  # Menor número de estimadores
)

# SVM con probabilidad calibrada
svm_opt = SVC(
    kernel='rbf', C=1.0, gamma='scale',
    class_weight='balanced', probability=True,
    random_state=42
)

models_opt = [('catb', catb_opt), ('xgb', xgb_opt), ('svm', svm_opt)]

# ===== Voting Classifier =====
fitted_opt, weights_opt = auc_weights(models_opt, X_train, y_train, X_val, y_val)
vote_opt = VotingClassifier(estimators=fitted_opt, voting='soft', weights=weights_opt)

# Entrenamiento del Voting Classifier
vote_opt.fit(X_train, y_train)
report("Voting CATB+XGB+SVM", vote_opt, X_val, y_val)

# Seleccionar el mejor modelo
best_model = vote_opt  # Si el VotingClassifier tiene buen rendimiento, puedes prescindir del StackingClassifier
best_model.fit(X_train, y_train)

scale_pos_weight=2.32
catb: AUC=0.9446
xgb: AUC=0.9414
svm: AUC=0.9443
Pesos: {'catb': 0.334, 'xgb': 0.332, 'svm': 0.334}
[Voting CATB+XGB+SVM] AUC=0.9486  F1=0.8214  Acc=0.9026


VotingClassifier(estimators=[('catb',
                              <catboost.core.CatBoostClassifier object at 0x79577a789e90>),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=None,
                                            grow_pol...
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('svm',
                              SVC(class_weight='balanced', probability=True,
                                  random_state=42))],
                 voting='soft',
                 weights=[0.33418560792180907, 0.33182853601305584,
                          0.3339858560651351])

## 8️⃣ Predicciones Finales en Test

In [26]:

y_pred_test = best_model.predict(X_test)
y_pred_proba_test = best_model.predict_proba(X_test)[:, 1]

# Crear submission
submission = pd.DataFrame({
    'patient_id': df_test_final['patient_id'],
    'has_diabetes': y_pred_test,
    'probability': y_pred_proba_test
})

# En tu notebook - al final
import pickle


pickle.dump(best_model, open('best_model.pkl', 'wb'))

import pickle

model = pickle.load(open('best_model.pkl', 'rb'))




In [23]:
print("\n" + "=" * 80)
print("GENERANDO PREDICCIONES EN TEST")
print("=" * 80)

print("\n Prediciendo...")
y_pred_test = best_model.predict(X_test)
y_pred_proba_test = best_model.predict_proba(X_test)[:, 1]

# Crear submission
submission = pd.DataFrame({
    'patient_id': df_test_final['patient_id'],
    'has_diabetes': y_pred_test,
    'probability': y_pred_proba_test
})

print(f"\n✅ {len(submission)} predicciones generadas")

print(f"\n📊 Distribución predicciones:")
dist = submission['has_diabetes'].value_counts()
print(f"   Negativo (0): {dist[0]} ({dist[0]/len(submission)*100:.1f}%)")
print(f"   Positivo (1): {dist[1]} ({dist[1]/len(submission)*100:.1f}%)")

print(f"\n📊 Probabilidades:")
print(f"   Media: {submission['probability'].mean():.4f}")
print(f"   Min: {submission['probability'].min():.4f}")
print(f"   Max: {submission['probability'].max():.4f}")

print(f"\n📋 PRIMERAS 10 PREDICCIONES:")
print(submission.head(10).to_string(index=False))
def parse_submission(id):
    id = str(id)
    length = len(id)
    return "patient_" + "0"* (5 - length) + id

submission["patient_id"] = submission["patient_id"].apply(parse_submission)
submission[["patient_id","has_diabetes"]].to_csv("submission.csv", index=False)
print(f"\nGuardado: submission.csv")
import joblib
scalers = [scaler_age, scaler_bmi, scaler_hba1c, scaler_glucose]

joblib.dump(best_model, "model.pkl")
joblib.dump(scalers, "scalers.pkl")


GENERANDO PREDICCIONES EN TEST

 Prediciendo...

✅ 300 predicciones generadas

📊 Distribución predicciones:
   Negativo (0): 218 (72.7%)
   Positivo (1): 82 (27.3%)

📊 Probabilidades:
   Media: 0.2887
   Min: 0.0003
   Max: 0.9994

📋 PRIMERAS 10 PREDICCIONES:
 patient_id  has_diabetes  probability
        139             0     0.010946
        252             0     0.005858
        259             0     0.009183
        335             1     0.956264
        699             0     0.001573
        977             0     0.033190
       1025             0     0.092884
       1145             0     0.028866
       1217             0     0.443703
       1235             1     0.858153

Guardado: submission.csv


['scalers.pkl']